## Carregamento da base

Nesta seção, iremos realizar o carregamento e validação básica da base.

Fonte dos dados: https://www.kaggle.com/datasets/mirichoi0218/insurance

In [3]:
import pandas as pd

df = pd.read_csv('insurance.csv')

print(df.head(5))

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [4]:
# Print da quantidade de linhas:

row_count = len(df)
print("Linhas:", row_count)

Linhas: 1338


### Verificação de inconsistências

In [5]:
# Verifica inconsistências
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

## Pré-processamento dos dados

Na seção abaixo, iremos realizar o mesmo pré processamento descrito na entrega do tech challenge da fase 1. Onde a base será normalizada e terá seus outliers removidos.

In [31]:
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

# Separa o dataframe original em dados categoricos e numericos
df_categoricos = df.select_dtypes(include=['object', 'category'])
df_numericos = df.select_dtypes(include=['number'])

# Inicializa OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Aplica o fit transform nos dados categoricos
one_hot_encoded = encoder.fit_transform(df_categoricos)

# Cria um DataFrame com os dados categoricos que tiveram o one hot encoding aplicado
encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out())

# Concatena o DataFrame de dados numericos com o DataFrame que contém o resultado do one hot encoding
df_result_one_hot_encoding = pd.concat([encoded_df, df_numericos], axis=1)

# Exibe as primeiras linhas para verificarmos o resultado
#print(df_result_one_hot_encoding.head(5))

# Exibe a quantidade de nulos gerados para validar se não foi criado um DataFrame com inconsistências
#print("Quantidade de nulos:\n", df_result_one_hot_encoding.isnull().sum())

# Printa a quantidade de linhas antes da remoção de outliers
row_count = len(df_result_one_hot_encoding)
#print("Row count:", row_count)

# Separa e printa as 5 ocorrências de maior valor para o IMC
top_bmi = df_result_one_hot_encoding.nlargest(5, 'bmi')
#print(top_bmi['bmi'])

# Calcula os percentis de Q1 e Q3 que serão utilizados no cálculo e a dispersão (IQR)
Q1 = df_result_one_hot_encoding['bmi'].quantile(0.25)
Q3 = df_result_one_hot_encoding['bmi'].quantile(0.75)
IQR = Q3 - Q1

# Define os limites de outlier
# Utilizaremos as formulas padrão de outliers, o resultado define quais valores serão os limites máximos para um valor ser considerado outlier ou não.
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove os outliers
df_no_outliers = df_result_one_hot_encoding[(df_result_one_hot_encoding['bmi'] <= upper_bound)]

# Separa e printa as 5 ocorrências de maior valor para o IMC
top_bmi = df_result_one_hot_encoding.nlargest(5, 'bmi')
#print(top_bmi['bmi'])

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Realiza o fit transform, normalizando as colunas
dados_normalizados = scaler.fit_transform(df_no_outliers)

# Converte o resultado do fit transform novamente em um data frame
df_normalizado = pd.DataFrame(dados_normalizados, columns=df_no_outliers.columns)

# Printa as informações normalizadas
#print(df_normalizado)

## Execução Random Forest sem algoritmo genético

Nesta seção, iremos realizar a execução do algoritmo de random forest da maneira padrão, sem selecionarmos os hiperparametros. Afim de coletarmos a pontuação R2 para posteriormente compararmos com performance do algoritmo genético.

In [113]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df_normalizado[['smoker_yes', 'smoker_no', 'age', 'children', 'bmi']]  
y = df_normalizado['charges'] 

model = RandomForestRegressor(random_state=42)

# Separa os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = r2_score(y_test, predictions)

print("Pontuacao R2 sem algoritmo genetico: ", score)

Pontuacao R2 sem algoritmo genetico:  0.8304680675627977


Como podemos observar acima, a pontuação R2 obtida da execução do algoritmo de Random Forest foi de: 0.8304680675627977


==================================================================================================================================================================================

## Execução do algoritmo Genetico

Na seção abaixo, iremos executar uma função com um algoritmo genético criado para otimizar os hiperparâmetros do algoritmo de random forest e coletarmos seus resultados.

In [114]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


def random_forest_genetic_algorithm(X, Y, generations=3, population_size=30, elitism=2, percent_childs=0.3):
    
    # Initializa população com individuos gerados com genes aleatorios
    def init_population():
        return [{
            'n_estimators': np.random.randint(10, 200),
            'max_depth': np.random.choice([None, np.random.randint(1, 100)]),
            'min_samples_split': np.random.randint(2, 30),
            'min_samples_leaf': np.random.randint(1, 30),
            'max_features': np.random.choice(['sqrt', 'log2'])
        } for _ in range(population_size)]

    # Avalia a aptidação dos individuos de uma população e a retorna uma lista rankeada em ordem decrescente
    def evaluate_population(population, X_train, X_test, y_train, y_test):
        scores = []
        for individual in population:
            model = RandomForestRegressor(
                n_estimators=individual['n_estimators'],
                max_depth=individual['max_depth'],
                min_samples_split=individual['min_samples_split'],
                min_samples_leaf=individual['min_samples_leaf'],
                max_features=individual['max_features'],
                random_state=42
            )
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            score = r2_score(y_test, predictions)
            scores.append((score, individual))

        return sorted(scores, key=lambda x: x[0], reverse=True)

    # Gera um filho a partir de dois pais, utilizando Crossover Uniforme
    def crossover(parent1, parent2):
        child = {}
        for key in parent1:
            if np.random.rand() > 0.5:
                child[key] = parent1[key]
            else:
                child[key] = parent2[key]
        return child
    
    # Aplica mutação 
    def mutate(individual, mutation_rate=0.3):
        if np.random.rand() < mutation_rate:
            # Randomly change one of the hyperparameters
            mutation = np.random.choice(list(individual.keys()))
            if mutation in ['n_estimators', 'min_samples_split', 'min_samples_leaf']:
              individual[mutation] = np.random.randint(10, 200) if mutation == 'n_estimators' else np.random.randint(2, 30) 
            elif mutation == 'max_depth':
                individual['max_depth'] = np.random.choice([None, np.random.randint(1, 100)])
            elif mutation == 'max_features':
                individual['max_features'] = np.random.choice(['sqrt', 'log2'])
        return individual

    
    # Divide os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Inicializa a primeira geração
    population = init_population()

    # Itera e gera cada geração até que o numero maximo de gerações seja atingido
    for n in range(generations):
        ranked_population = evaluate_population(population, X_train, X_test, y_train, y_test)
        #print("Populacao rankeada: ", ranked_population)

        # Seleciona os melhores individuos de acordo com o elitismo parametrizado
        top_scores = [ind[0] for ind in ranked_population[:elitism]]
        top_individuals = [ind[1] for ind in ranked_population[:elitism]]
        print(f"Melhores scores da geracao {n}: ", top_scores)
        #print(f"Melhores individuos da geracao {n}: ", top_individuals)

        # Gera uma nova população aleatoria
        new_population = init_population()

        #print("populacao inicial: ", new_population)
        # Substitui % da populacao com individuos gerados a partir do crossover dos melhores selecionados a partir da população anterior
        i = 0
        while i <= population_size * percent_childs - 1:
            parent1, parent2 = np.random.choice(top_individuals, 2, replace=False)
            #print ("parent 1: ", parent1)
            #print ("parent 2: ", parent2)
            child = crossover(parent1, parent2)
            child = mutate(child)
            #print ("child: ", child)
            #print ("\n")
            new_population[i] = child
            i += 1
            
        #print("populacao mista: ", new_population)

        population = new_population


    # Avalia a população final e recebe a lista rankeada
    final_ranking = evaluate_population(population, X_train, X_test, y_train, y_test)

    # Seleciona as informações do melhor individuo
    best_score, best_hyperparameters = final_ranking[0]

    # Printa o score final e hiperaparametros utilizados
    print("\n")
    print(f"Score R2 do melhor individuo da geração final: {best_score}")
    print(f"Hiperparametros do melhor individuo da geração final : {best_hyperparameters}")

    return best_hyperparameters


In [116]:
random_forest_genetic_algorithm(X, y, generations=10, population_size=30, elitism=4, percent_childs=0.5)

Melhores scores da geracao 0:  [0.8675690460193384, 0.8661600446374275, 0.8654235137621262, 0.8653370610295994]
Melhores scores da geracao 1:  [0.8675690460193384, 0.8675690460193384, 0.8675690460193384, 0.8675690460193384]
Melhores scores da geracao 2:  [0.8677527740435681, 0.8675690460193384, 0.8675690460193384, 0.8675690460193384]
Melhores scores da geracao 3:  [0.8695267420820865, 0.8677527740435681, 0.8675690460193384, 0.8675690460193384]
Melhores scores da geracao 4:  [0.8695267420820865, 0.8677527740435681, 0.8677527740435681, 0.8675690460193384]
Melhores scores da geracao 5:  [0.8695617793541377, 0.8695617793541377, 0.8677527740435681, 0.8677527740435681]
Melhores scores da geracao 6:  [0.8695617793541377, 0.8695617793541377, 0.8695617793541377, 0.8695617793541377]
Melhores scores da geracao 7:  [0.8708333955292237, 0.8695617793541377, 0.8695617793541377, 0.8695617793541377]
Melhores scores da geracao 8:  [0.8708333955292237, 0.8708333955292237, 0.8697980291709999, 0.8695617793

{'n_estimators': 52,
 'max_depth': 5,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': np.str_('log2')}